In [ ]:

import os
import torch
from datasets import load_dataset
from transformers import (
    AutoProcessor,
    LlavaForConditionalGeneration,
    TrainingArguments,
    Trainer,
)
from peft import prepare_model_for_kbit_training, get_peft_model, LoraConfig
from transformers import BitsAndBytesConfig


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving dataset_uni.jsonl to dataset_uni.jsonl


In [ ]:
import zipfile
import os

with zipfile.ZipFile("images.zip", "r") as zip_ref:
    zip_ref.extractall(".")

# Vérifie que c’est bien là
os.listdir("images")


['Math_Free_Response',
 'Physics_Free_Response',
 'Math',
 'Biology',
 'Chemistry',
 'Biology_Free_Response',
 'Chemistry_extra',
 'Chemistry_Free_Response',
 'Physics']

In [ ]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 100.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [ ]:
#  Chemin du dataset unifié
DATA_PATH = "dataset_uni.jsonl"
MODEL_NAME = "llava-hf/llava-1.5-7b-hf"

#  Chargement du dataset
dataset = load_dataset("json", data_files=DATA_PATH, split="train")

In [ ]:
processor = AutoProcessor.from_pretrained(MODEL_NAME)

model = LlavaForConditionalGeneration.from_pretrained(
    MODEL_NAME,
    load_in_4bit=True,
    torch_dtype=torch.float16,

)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quan

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.18G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

In [ ]:
#  Préparer le modèle LoRA pour fine-tuning
model = prepare_model_for_kbit_training(model)

# Configuration LoRA optimisée
peft_config = LoraConfig(
    r=64,  # Augmenté de 16 à 64 pour plus de capacité
    lora_alpha=128,  # Augmenté de 32 à 128 (ratio 2:1 avec r)
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],  # Plus de modules ciblés
    lora_dropout=0.1,  # Augmenté de 0.05 à 0.1 pour éviter l'overfitting
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, peft_config)

In [ ]:
# Arguments d'entraînement optimisés
training_args = TrainingArguments(
    output_dir="./llava-finetuned",
    per_device_train_batch_size=2,  # Augmenté de 1 à 2
    gradient_accumulation_steps=8,  # Augmenté de 4 à 8
    num_train_epochs=5,  # Augmenté de 3 à 5
    learning_rate=5e-5,  # Augmenté de 2e-5 à 5e-5
    warmup_steps=100,  # Ajouté warmup
    weight_decay=0.01,  # Ajouté weight decay
    lr_scheduler_type="cosine",  # Scheduler cosine
    fp16=True,
    logging_steps=5,  # Réduit de 10 à 5 pour plus de logs
    save_strategy="steps",  # Changé de "epoch" à "steps"
    save_steps=200,  # Sauvegarde tous les 200 steps
    save_total_limit=3,  # Augmenté de 1 à 3
    dataloader_num_workers=2,  # Ajouté pour parallélisation
    remove_unused_columns=False,
    report_to="none"
)

In [ ]:
def clean_path(example):
    example["image"] = example["image"].replace("\\", "/")
    return example

# Appliquer le nettoyage
dataset = dataset.map(clean_path)

Map:   0%|          | 0/1040 [00:00<?, ? examples/s]

In [ ]:
from PIL import Image
import os
import torch

def preprocess(example):
    image_path = example["image"]

    # Vérifier si l'image existe avec une extension .png ou .jpg
    if not os.path.exists(image_path):  # Si le fichier n'existe pas
        # Essayer d'ajuster l'extension pour vérifier si l'image existe avec l'extension .jpg
        image_path = image_path.replace(".png", ".jpg")
        if not os.path.exists(image_path):  # Si le fichier avec l'extension .jpg n'existe toujours pas
            raise FileNotFoundError(f"Image not found at {image_path}")

    # Ouvrir l'image en RGB
    image = Image.open(image_path).convert("RGB")

    # Resize image to the expected input size for the LLaVA model
    image = image.resize((224, 224))

    # **Add the special image token `<image>` to the prompt:**
    prompt = f"Question: {example['question']}\n<image>\nAnswer:"

    # Créer le texte complet avec la réponse pour les labels
    full_text = f"Question: {example['question']}\n<image>\nAnswer: {example['answer']}"

    # Encoder le prompt (sans la réponse) pour les input_ids
    prompt_encoding = processor(text=prompt, images=image, return_tensors="pt", padding=False, truncation=True)

    # Encoder le texte complet pour créer les labels
    full_encoding = processor(text=full_text, images=image, return_tensors="pt", padding=False, truncation=True)

    # Créer les labels : -100 pour les tokens du prompt, les vrais tokens pour la réponse
    labels = full_encoding["input_ids"].clone()
    prompt_length = prompt_encoding["input_ids"].shape[1]
    labels[0, :prompt_length] = -100  # Ignorer les tokens du prompt dans le calcul de la loss

    return {
        "input_ids": full_encoding["input_ids"].squeeze(0).tolist(),
        "attention_mask": full_encoding["attention_mask"].squeeze(0).tolist(),
        "pixel_values": full_encoding["pixel_values"].squeeze(0).tolist(),
        "labels": labels.squeeze(0).tolist()
    }

#  Appliquer la fonction
processed_dataset = dataset.map(preprocess, remove_columns=["image", "question", "answer"])


Map:   0%|          | 0/1040 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForLanguageModeling

class LlavaDataCollator:
    def __init__(self, processor):
        self.processor = processor

    def __call__(self, features):
        # Trouver la longueur maximale dans le batch
        max_length = max(len(feature["input_ids"]) for feature in features)

        batch = {}
        for key in ["input_ids", "attention_mask", "labels"]:
            batch[key] = []
            for feature in features:
                # Convertir en tensor si c'est une liste
                seq = feature[key]
                if isinstance(seq, list):
                    seq = torch.tensor(seq)

                # Pad la séquence à la longueur maximale
                if len(seq) < max_length:
                    if key == "labels":
                        # Pour les labels, pad avec -100
                        padding = torch.full((max_length - len(seq),), -100, dtype=seq.dtype)
                    else:
                        # Pour input_ids et attention_mask, pad avec 0
                        padding = torch.zeros(max_length - len(seq), dtype=seq.dtype)
                    seq = torch.cat([seq, padding])
                batch[key].append(seq)
            batch[key] = torch.stack(batch[key])

        # Gérer pixel_values (même taille pour toutes les images)
        pixel_values = []
        for feature in features:
            pv = feature["pixel_values"]
            if isinstance(pv, list):
                pv = torch.tensor(pv)
            pixel_values.append(pv)
        batch["pixel_values"] = torch.stack(pixel_values)

        return batch

# Utiliser le data collator personnalisé
data_collator = LlavaDataCollator(processor)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=processed_dataset,
    data_collator=data_collator  # Ajout du data collator
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
#  Lancement de l'entraînement
trainer.train()
trainer.save_model("llava_finetune-VF")


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:457: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint:

Step,Training Loss
5,2.703700
10,2.441300
15,2.690900
20,2.134200
25,1.690600
30,1.564100
35,1.577800
40,1.451000
45,1.334200
50,1.238500


Step,Training Loss
5,2.703700
10,2.441300
15,2.690900
20,2.134200
25,1.690600
30,1.564100
35,1.577800
40,1.451000
45,1.334200
50,1.238500


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


In [ ]:
import json
import torch
from transformers import AutoProcessor, LlavaForConditionalGeneration
from peft import PeftModel
from collections import defaultdict
import time
import gc
import os

def evaluate_llava_full(model_path, unified_dataset_path):
    print("🚀 ÉVALUATION COMPLÈTE DU MODÈLE LLAVA")
    print("=" * 60)

    # Vérifier GPU
    print(f"GPU: {torch.cuda.get_device_name() if torch.cuda.is_available() else 'CPU'}")
    if torch.cuda.is_available():
        print(f"Mémoire GPU totale: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

    MODEL_NAME = "llava-hf/llava-1.5-7b-hf"
    processor = AutoProcessor.from_pretrained(MODEL_NAME)

    print("\n📥 Chargement du modèle...")
    model = LlavaForConditionalGeneration.from_pretrained(
        MODEL_NAME,
        load_in_4bit=True,
        torch_dtype=torch.float16,
        device_map="auto"
    )
    model = PeftModel.from_pretrained(model, model_path)
    model.eval()
    print("✅ Modèle chargé!")

    # Charger le dataset complet
    print("\n📂 Chargement du dataset complet...")
    dataset = []
    with open(unified_dataset_path, "r") as f:
        for line_num, line in enumerate(f):
            if line.strip():
                try:
                    entry = json.loads(line)
                    if "question" in entry and "answer" in entry:
                        category = detect_category(entry["question"])
                        dataset.append({
                            "question": entry["question"],
                            "answer": entry["answer"].strip(),
                            "category": category
                        })
                except json.JSONDecodeError:
                    continue

    print(f"✅ Dataset complet chargé: {len(dataset)} questions")

    # Séparer par type et compter par catégorie
    mc_by_category = defaultdict(list)
    fr_by_category = defaultdict(list)

    for item in dataset:
        ans = item["answer"]
        if len(ans) == 1 and ans.lower() in "abcde":
            mc_by_category[item["category"]].append(item)
        else:
            fr_by_category[item["category"]].append(item)

    # Afficher la répartition
    print("\n📊 RÉPARTITION DU DATASET COMPLET")
    print("=" * 50)

    print("\n🔤 Multiple Choice (MC):")
    mc_total = 0
    for category in sorted(mc_by_category.keys()):
        count = len(mc_by_category[category])
        mc_total += count
        print(f"  {category:12}: {count:4d} questions")
    print(f"  {'TOTAL MC':12}: {mc_total:4d} questions")

    print("\n✍️  Free Response (FR):")
    fr_total = 0
    for category in sorted(fr_by_category.keys()):
        count = len(fr_by_category[category])
        fr_total += count
        print(f"  {category:12}: {count:4d} questions")
    print(f"  {'TOTAL FR':12}: {fr_total:4d} questions")

    total_questions = mc_total + fr_total
    print(f"\n📈 TOTAL GÉNÉRAL: {total_questions} questions")

    # Estimation du temps
    estimated_time = total_questions * 2  # ~2 secondes par question
    print(f"⏱️  Temps estimé: {estimated_time // 60} minutes")

    # Préparer toutes les questions
    all_mc_questions = []
    all_fr_questions = []

    for category in mc_by_category:
        all_mc_questions.extend(mc_by_category[category])

    for category in fr_by_category:
        all_fr_questions.extend(fr_by_category[category])

    # Initialiser les résultats
    results = {
        "MC": {"total": defaultdict(int), "correct": defaultdict(int)},
        "FR": {"total": defaultdict(int), "correct": defaultdict(int)},
    }

    # Compter le total par catégorie
    for item in all_mc_questions:
        results["MC"]["total"][item["category"]] += 1
    for item in all_fr_questions:
        results["FR"]["total"][item["category"]] += 1

    def evaluate_question(question, expected_answer):
        prompt = f"Question: {question}\nAnswer:"
        inputs = processor(text=prompt, return_tensors="pt").to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                input_ids=inputs['input_ids'],
                attention_mask=inputs['attention_mask'],
                max_new_tokens=50,
                do_sample=False,
                num_return_sequences=1,
                pad_token_id=processor.tokenizer.eos_token_id
            )

        generated = processor.tokenizer.decode(outputs[0], skip_special_tokens=True)
        generated = generated.replace(prompt, "").strip()
        return generated.split('\n')[0]

    def check_answer(generated, expected):
        generated = generated.lower().strip()
        expected = expected.lower().strip()

        if len(expected) == 1 and expected in "abcde":
            return expected in generated[:5]
        else:
            if expected.replace('.', '', 1).isdigit():
                try:
                    gen_num = float(''.join(c for c in generated if c.isdigit() or c in '.-'))
                    exp_num = float(expected)
                    return abs(gen_num - exp_num) < 0.1
                except:
                    return False
            return expected in generated or generated in expected

    # Variables pour le suivi
    start_time = time.time()
    errors = 0

    # ÉVALUATION MULTIPLE CHOICE
    print("\n" + "="*60)
    print("🔤 ÉVALUATION MULTIPLE CHOICE")
    print("="*60)

    mc_start_time = time.time()

    for i, ex in enumerate(all_mc_questions):
        # Progression détaillée
        if i % 25 == 0:
            elapsed = time.time() - mc_start_time
            progress = (i / len(all_mc_questions)) * 100
            eta = (elapsed / (i + 1)) * (len(all_mc_questions) - i) if i > 0 else 0

            print(f"📊 MC Progress: {i:3d}/{len(all_mc_questions)} ({progress:5.1f}%) | "
                  f"Temps: {elapsed:5.1f}s | ETA: {eta:5.1f}s")

            # Nettoyage mémoire GPU
            torch.cuda.empty_cache()
            gc.collect()

        q = ex["question"]
        expected = ex["answer"]
        category = ex["category"]

        try:
            generated = evaluate_question(q, expected)
            correct = check_answer(generated, expected)

            if correct:
                results["MC"]["correct"][category] += 1

        except Exception as e:
            errors += 1
            if errors % 10 == 1:  # Afficher les erreurs occasionnellement
                print(f"⚠️  Erreur #{errors}: {str(e)[:50]}...")
            continue

    mc_duration = time.time() - mc_start_time
    print(f"✅ MC terminé en {mc_duration:.1f}s")

    # ÉVALUATION FREE RESPONSE
    print("\n" + "="*60)
    print("✍️  ÉVALUATION FREE RESPONSE")
    print("="*60)

    fr_start_time = time.time()

    for i, ex in enumerate(all_fr_questions):
        # Progression détaillée
        if i % 25 == 0:
            elapsed = time.time() - fr_start_time
            progress = (i / len(all_fr_questions)) * 100
            eta = (elapsed / (i + 1)) * (len(all_fr_questions) - i) if i > 0 else 0

            print(f"📊 FR Progress: {i:3d}/{len(all_fr_questions)} ({progress:5.1f}%) | "
                  f"Temps: {elapsed:5.1f}s | ETA: {eta:5.1f}s")

            # Nettoyage mémoire GPU
            torch.cuda.empty_cache()
            gc.collect()

        q = ex["question"]
        expected = ex["answer"]
        category = ex["category"]

        try:
            generated = evaluate_question(q, expected)
            correct = check_answer(generated, expected)

            if correct:
                results["FR"]["correct"][category] += 1

        except Exception as e:
            errors += 1
            if errors % 10 == 1:
                print(f"⚠️  Erreur #{errors}: {str(e)[:50]}...")
            continue

    fr_duration = time.time() - fr_start_time
    print(f"✅ FR terminé en {fr_duration:.1f}s")

    # AFFICHAGE DES RÉSULTATS
    total_duration = time.time() - start_time

    print("\n" + "="*80)
    print("📊 RÉSULTATS FINAUX - ÉVALUATION COMPLÈTE")
    print("="*80)

    # Obtenir toutes les catégories
    all_categories = set()
    all_categories.update(results["MC"]["total"].keys())
    all_categories.update(results["FR"]["total"].keys())
    all_categories = sorted(all_categories)

    # Tableau détaillé
    print(f"\n{'Catégorie':12} | {'MC (Correct/Total)':18} | {'Acc. MC':8} | {'FR (Correct/Total)':18} | {'Acc. FR':8} | {'Total':12}")
    print("-" * 95)

    for category in all_categories:
        mc_correct = results["MC"]["correct"][category]
        mc_total = results["MC"]["total"][category]
        fr_correct = results["FR"]["correct"][category]
        fr_total = results["FR"]["total"][category]

        mc_acc = (mc_correct / mc_total) * 100 if mc_total > 0 else 0
        fr_acc = (fr_correct / fr_total) * 100 if fr_total > 0 else 0

        cat_total_correct = mc_correct + fr_correct
        cat_total_questions = mc_total + fr_total
        cat_acc = (cat_total_correct / cat_total_questions) * 100 if cat_total_questions > 0 else 0

        print(f"{category:12} | {mc_correct:3d}/{mc_total:3d} ({mc_acc:5.1f}%) | {mc_acc:6.1f}% | "
              f"{fr_correct:3d}/{fr_total:3d} ({fr_acc:5.1f}%) | {fr_acc:6.1f}% | {cat_acc:6.1f}%")

    # Totaux
    print("-" * 95)
    mc_total_correct = sum(results["MC"]["correct"].values())
    mc_total_questions = sum(results["MC"]["total"].values())
    fr_total_correct = sum(results["FR"]["correct"].values())
    fr_total_questions = sum(results["FR"]["total"].values())

    mc_accuracy = (mc_total_correct / mc_total_questions) * 100 if mc_total_questions > 0 else 0
    fr_accuracy = (fr_total_correct / fr_total_questions) * 100 if fr_total_questions > 0 else 0

    overall_correct = mc_total_correct + fr_total_correct
    overall_total = mc_total_questions + fr_total_questions
    overall_accuracy = (overall_correct / overall_total) * 100 if overall_total > 0 else 0

    print(f"{'TOTAL':12} | {mc_total_correct:3d}/{mc_total_questions:3d} ({mc_accuracy:5.1f}%) | {mc_accuracy:6.1f}% | "
          f"{fr_total_correct:3d}/{fr_total_questions:3d} ({fr_accuracy:5.1f}%) | {fr_accuracy:6.1f}% | {overall_accuracy:6.1f}%")

    # Résumé final
    print("\n" + "="*80)
    print("🎯 RÉSUMÉ FINAL")
    print("="*80)

    print(f"\n📊 PERFORMANCES GLOBALES:")
    print(f"   • Multiple Choice: {mc_total_correct:3d}/{mc_total_questions:3d} = {mc_accuracy:5.1f}%")
    print(f"   • Free Response:   {fr_total_correct:3d}/{fr_total_questions:3d} = {fr_accuracy:5.1f}%")
    print(f"   • PRÉCISION GLOBALE: {overall_correct:3d}/{overall_total:3d} = {overall_accuracy:5.1f}%")

    print(f"\n🔬 ANALYSE PAR MATIÈRE:")
    for category in all_categories:
        mc_correct = results["MC"]["correct"][category]
        mc_total = results["MC"]["total"][category]
        fr_correct = results["FR"]["correct"][category]
        fr_total = results["FR"]["total"][category]

        cat_total_correct = mc_correct + fr_correct
        cat_total_questions = mc_total + fr_total
        cat_accuracy = (cat_total_correct / cat_total_questions) * 100 if cat_total_questions > 0 else 0

        print(f"   • {category:10}: {cat_total_correct:3d}/{cat_total_questions:3d} = {cat_accuracy:5.1f}%")

    print(f"\n⏱️  STATISTIQUES D'EXÉCUTION:")
    print(f"   • Durée totale: {total_duration:.1f}s ({total_duration/60:.1f} min)")
    print(f"   • Temps par question: {total_duration/total_questions:.2f}s")
    print(f"   • Erreurs: {errors}")

    print(f"\n✅ ÉVALUATION TERMINÉE!")

    return {
        "mc_accuracy": mc_accuracy,
        "fr_accuracy": fr_accuracy,
        "overall_accuracy": overall_accuracy,
        "detailed_results": results,
        "execution_time": total_duration,
        "total_questions": total_questions,
        "errors": errors
    }

def detect_category(question):
    """Détecte la catégorie d'une question basée sur des mots-clés"""
    question_lower = question.lower()

    math_keywords = ['calculate', 'equation', 'solve', 'number', 'sum', 'multiply', 'divide', 'fraction', 'percentage', 'mathematics', 'algebra', 'geometry']
    bio_keywords = ['biology', 'cell', 'organism', 'species', 'dna', 'protein', 'evolution', 'ecosystem', 'photosynthesis', 'genetics', 'anatomy']
    physics_keywords = ['physics', 'force', 'energy', 'motion', 'velocity', 'acceleration', 'mass', 'gravity', 'electromagnetic', 'thermodynamics']
    chemistry_keywords = ['chemistry', 'molecule', 'atom', 'reaction', 'compound', 'element', 'chemical', 'bond', 'solution', 'acid', 'base']

    if any(keyword in question_lower for keyword in math_keywords):
        return "Math"
    elif any(keyword in question_lower for keyword in bio_keywords):
        return "Biology"
    elif any(keyword in question_lower for keyword in physics_keywords):
        return "Physics"
    elif any(keyword in question_lower for keyword in chemistry_keywords):
        return "Chemistry"
    else:
        return "General"

if __name__ == "__main__":
    model_path = "/content/drive/MyDrive/Tyda/llava_finetune-VF"
    unified_dataset_path = "/content/dataset_uni.jsonl"

    print("🚀 LANCEMENT DE L'ÉVALUATION COMPLÈTE")
    print(f"📂 Modèle: {model_path}")
    print(f"📂 Dataset: {unified_dataset_path}")

    # Vérifier les chemins
    if not os.path.exists(model_path):
        print(f"❌ Modèle introuvable: {model_path}")
        exit(1)

    if not os.path.exists(unified_dataset_path):
        print(f"❌ Dataset introuvable: {unified_dataset_path}")
        exit(1)

    # Lancer l'évaluation complète
    results = evaluate_llava_full(model_path, unified_dataset_path)

🚀 LANCEMENT DE L'ÉVALUATION COMPLÈTE
📂 Modèle: /content/drive/MyDrive/Tyda/llava_finetune-VF
📂 Dataset: /content/dataset_uni.jsonl
🚀 ÉVALUATION COMPLÈTE DU MODÈLE LLAVA
GPU: NVIDIA A100-SXM4-40GB
Mémoire GPU totale: 42.5 GB


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.



📥 Chargement du modèle...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

✅ Modèle chargé!

📂 Chargement du dataset complet...
✅ Dataset complet chargé: 1040 questions

📊 RÉPARTITION DU DATASET COMPLET

🔤 Multiple Choice (MC):
  Biology     :   63 questions
  Chemistry   :   87 questions
  General     :  101 questions
  Math        :   89 questions
  Physics     :   42 questions
  TOTAL MC    :  382 questions

✍️  Free Response (FR):
  Biology     :   53 questions
  Chemistry   :   65 questions
  General     :  219 questions
  Math        :  170 questions
  Physics     :  151 questions
  TOTAL FR    :  658 questions

📈 TOTAL GÉNÉRAL: 1040 questions
⏱️  Temps estimé: 34 minutes

🔤 ÉVALUATION MULTIPLE CHOICE
📊 MC Progress:   0/382 (  0.0%) | Temps:   0.0s | ETA:   0.0s
📊 MC Progress:  25/382 (  6.5%) | Temps: 122.3s | ETA: 1679.8s
📊 MC Progress:  50/382 ( 13.1%) | Temps: 234.0s | ETA: 1523.6s
📊 MC Progress:  75/382 ( 19.6%) | Temps: 356.2s | ETA: 1438.9s
📊 MC Progress: 100/382 ( 26.2%) | Temps: 483.6s | ETA: 1350.2s
📊 MC Progress: 125/382 ( 32.7%) | Temps: 573

In [ ]:
import json
import torch
from transformers import AutoProcessor, LlavaForConditionalGeneration
from peft import PeftModel
from collections import defaultdict
import time
import os

def debug_and_evaluate_llava(model_path, unified_dataset_path):
    print("🔍 DIAGNOSTIC ET ÉVALUATION DEBUG")
    print("=" * 50)

    # Vérifier GPU
    print(f"GPU disponible: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"GPU actuel: {torch.cuda.get_device_name()}")
        print(f"Mémoire GPU: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

    MODEL_NAME = "llava-hf/llava-1.5-7b-hf"
    processor = AutoProcessor.from_pretrained(MODEL_NAME)

    print("\n📥 Chargement du modèle...")
    try:
        model = LlavaForConditionalGeneration.from_pretrained(
            MODEL_NAME,
            load_in_4bit=True,
            torch_dtype=torch.float16,
            device_map="auto"
        )
        model = PeftModel.from_pretrained(model, model_path)
        model.eval()
        print("✅ Modèle chargé avec succès!")
    except Exception as e:
        print(f"❌ Erreur lors du chargement: {e}")
        return

    # Test rapide du modèle
    print("\n🧪 Test rapide du modèle...")
    try:
        test_prompt = "Question: What is 2+2?\nAnswer:"
        inputs = processor(text=test_prompt, return_tensors="pt").to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                input_ids=inputs['input_ids'],
                attention_mask=inputs['attention_mask'],
                max_new_tokens=10,
                do_sample=False,
                pad_token_id=processor.tokenizer.eos_token_id
            )

        generated = processor.tokenizer.decode(outputs[0], skip_special_tokens=True)
        print(f"✅ Test réussi! Réponse: {generated}")

    except Exception as e:
        print(f"❌ Erreur lors du test: {e}")
        return

    # Charger le dataset
    print("\n📂 Chargement du dataset...")
    dataset = []
    try:
        with open(unified_dataset_path, "r") as f:
            for line_num, line in enumerate(f):
                if line.strip():
                    try:
                        entry = json.loads(line)
                        if "question" in entry and "answer" in entry:
                            category = detect_category(entry["question"])
                            dataset.append({
                                "question": entry["question"],
                                "answer": entry["answer"].strip(),
                                "category": category
                            })
                    except json.JSONDecodeError as e:
                        print(f"⚠️ Erreur JSON ligne {line_num + 1}: {e}")
                        continue

        print(f"✅ Dataset chargé: {len(dataset)} questions")

    except Exception as e:
        print(f"❌ Erreur lors du chargement du dataset: {e}")
        return

    # Séparer par type
    mc_questions = []
    fr_questions = []

    for item in dataset:
        ans = item["answer"]
        if len(ans) == 1 and ans.lower() in "abcde":
            mc_questions.append(item)
        else:
            fr_questions.append(item)

    print(f"📊 Répartition: {len(mc_questions)} MC, {len(fr_questions)} FR")

    # Fonction d'évaluation avec debug
    def evaluate_question_debug(question, expected_answer, question_num=0):
        try:
            if question_num % 10 == 0:  # Debug tous les 10 questions
                print(f"🔄 Question {question_num}: {question[:50]}...")

            prompt = f"Question: {question}\nAnswer:"
            inputs = processor(text=prompt, return_tensors="pt").to(model.device)

            with torch.no_grad():
                outputs = model.generate(
                    input_ids=inputs['input_ids'],
                    attention_mask=inputs['attention_mask'],
                    max_new_tokens=50,
                    do_sample=False,
                    num_return_sequences=1,
                    pad_token_id=processor.tokenizer.eos_token_id
                )

            generated = processor.tokenizer.decode(outputs[0], skip_special_tokens=True)
            generated = generated.replace(prompt, "").strip()
            return generated.split('\n')[0]

        except Exception as e:
            print(f"❌ Erreur question {question_num}: {e}")
            return "ERROR"

    def check_answer(generated, expected):
        generated = generated.lower().strip()
        expected = expected.lower().strip()

        if len(expected) == 1 and expected in "abcde":
            return expected in generated[:5]
        else:
            if expected.replace('.', '', 1).isdigit():
                try:
                    gen_num = float(''.join(c for c in generated if c.isdigit() or c in '.-'))
                    exp_num = float(expected)
                    return abs(gen_num - exp_num) < 0.1
                except:
                    return False
            return expected in generated or generated in expected

    # Évaluation avec limite pour test
    print("\n🚀 Démarrage évaluation (mode debug - 50 questions max par type)")

    # Limiter pour le debug
    test_mc = mc_questions[:min(50, len(mc_questions))]
    test_fr = fr_questions[:min(50, len(fr_questions))]

    results = {
        "MC": {"total": defaultdict(int), "correct": defaultdict(int)},
        "FR": {"total": defaultdict(int), "correct": defaultdict(int)},
    }

    # Évaluation MC
    print(f"\n📝 Évaluation MC ({len(test_mc)} questions)...")
    mc_correct = 0

    for i, ex in enumerate(test_mc):
        q = ex["question"]
        expected = ex["answer"]
        category = ex["category"]

        results["MC"]["total"][category] += 1

        generated = evaluate_question_debug(q, expected, i)
        if generated != "ERROR":
            correct = check_answer(generated, expected)
            if correct:
                results["MC"]["correct"][category] += 1
                mc_correct += 1

            # Afficher quelques exemples
            if i < 5:
                print(f"  Ex {i+1}: {q[:30]}... -> {generated[:20]}... ({'✅' if correct else '❌'})")

        # Vider le cache GPU périodiquement
        if i % 10 == 0:
            torch.cuda.empty_cache()

    # Évaluation FR
    print(f"\n📝 Évaluation FR ({len(test_fr)} questions)...")
    fr_correct = 0

    for i, ex in enumerate(test_fr):
        q = ex["question"]
        expected = ex["answer"]
        category = ex["category"]

        results["FR"]["total"][category] += 1

        generated = evaluate_question_debug(q, expected, i)
        if generated != "ERROR":
            correct = check_answer(generated, expected)
            if correct:
                results["FR"]["correct"][category] += 1
                fr_correct += 1

            # Afficher quelques exemples
            if i < 5:
                print(f"  Ex {i+1}: {q[:30]}... -> {generated[:20]}... ({'✅' if correct else '❌'})")

        # Vider le cache GPU périodiquement
        if i % 10 == 0:
            torch.cuda.empty_cache()

    # Résultats
    print("\n" + "="*60)
    print("RÉSULTATS DEBUG")
    print("="*60)

    mc_total = len(test_mc)
    fr_total = len(test_fr)

    mc_accuracy = (mc_correct / mc_total) * 100 if mc_total > 0 else 0
    fr_accuracy = (fr_correct / fr_total) * 100 if fr_total > 0 else 0
    overall_accuracy = ((mc_correct + fr_correct) / (mc_total + fr_total)) * 100 if (mc_total + fr_total) > 0 else 0

    print(f"Multiple Choice: {mc_correct}/{mc_total} ({mc_accuracy:.1f}%)")
    print(f"Free Response:  {fr_correct}/{fr_total} ({fr_accuracy:.1f}%)")
    print(f"GLOBAL:         {mc_correct + fr_correct}/{mc_total + fr_total} ({overall_accuracy:.1f}%)")

    # Si le test fonctionne, proposer l'évaluation complète
    if mc_accuracy > 0 or fr_accuracy > 0:
        print("\n✅ Le modèle fonctionne! Vous pouvez lancer l'évaluation complète.")
        return True
    else:
        print("\n❌ Problème détecté. Vérifiez les résultats ci-dessus.")
        return False

def detect_category(question):
    """Détecte la catégorie d'une question basée sur des mots-clés"""
    question_lower = question.lower()

    math_keywords = ['calculate', 'equation', 'solve', 'number', 'sum', 'multiply', 'divide', 'fraction', 'percentage', 'mathematics', 'algebra', 'geometry']
    bio_keywords = ['biology', 'cell', 'organism', 'species', 'dna', 'protein', 'evolution', 'ecosystem', 'photosynthesis', 'genetics', 'anatomy']
    physics_keywords = ['physics', 'force', 'energy', 'motion', 'velocity', 'acceleration', 'mass', 'gravity', 'electromagnetic', 'thermodynamics']
    chemistry_keywords = ['chemistry', 'molecule', 'atom', 'reaction', 'compound', 'element', 'chemical', 'bond', 'solution', 'acid', 'base']

    if any(keyword in question_lower for keyword in math_keywords):
        return "Math"
    elif any(keyword in question_lower for keyword in bio_keywords):
        return "Biology"
    elif any(keyword in question_lower for keyword in physics_keywords):
        return "Physics"
    elif any(keyword in question_lower for keyword in chemistry_keywords):
        return "Chemistry"
    else:
        return "General"

if __name__ == "__main__":
    # Vérifier les chemins
    model_path = "/content/drive/MyDrive/Tyda/llava_finetune-VF"
    unified_dataset_path = "/content/dataset_uni.jsonl"

    print("🔍 Vérification des chemins...")
    if not os.path.exists(model_path):
        print(f"❌ Modèle introuvable: {model_path}")
        # Chercher dans le dossier
        tyda_path = "/content/drive/MyDrive/Tyda"
        if os.path.exists(tyda_path):
            print(f"📁 Contenu du dossier Tyda:")
            for item in os.listdir(tyda_path):
                print(f"  - {item}")
        exit(1)

    if not os.path.exists(unified_dataset_path):
        print(f"❌ Dataset introuvable: {unified_dataset_path}")
        exit(1)

    print("✅ Chemins vérifiés")

    # Lancer le debug
    debug_and_evaluate_llava(model_path, unified_dataset_path)

🔍 Vérification des chemins...
✅ Chemins vérifiés
🔍 DIAGNOSTIC ET ÉVALUATION DEBUG
GPU disponible: True
GPU actuel: NVIDIA A100-SXM4-40GB
Mémoire GPU: 42.5 GB


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.



📥 Chargement du modèle...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

✅ Modèle chargé avec succès!

🧪 Test rapide du modèle...
✅ Test réussi! Réponse: Question: What is 2+2?
Answer: 2+2 is 4.

📂 Chargement du dataset...
✅ Dataset chargé: 1040 questions
📊 Répartition: 382 MC, 658 FR

🚀 Démarrage évaluation (mode debug - 50 questions max par type)

📝 Évaluation MC (50 questions)...
🔄 Question 0: At which value of x is f continuous but not differ...
  Ex 1: At which value of x is f conti... -> D:f(x) is continuous... (✅)
  Ex 2: The table shows values of $f'$... -> B: $x<1$ or $x>5$... (✅)
  Ex 3: Shown is a graph of $f''$, the... -> E: f has inflection ... (✅)
  Ex 4: Shown in the diagram is a grap... -> B: $f(1) = \frac{13}... (✅)
  Ex 5: Shown is a graph of $f'$, the ... -> D: $f$ is increasing... (✅)
🔄 Question 10: The graph of a function, $f$, is shown above. Let ...
🔄 Question 20: The prices, in thousands of dollars, of 304 homes ...
🔄 Question 30: In the northern United States, schools are sometim...
🔄 Question 40: For the histogram pictured above, w

In [ ]:
import os

from google.colab import drive
drive.mount('/content/drive')




Mounted at /content/drive


In [ ]:
import secrets
print(secrets.token_urlsafe(32))


bnX1VvZn1v4XAkISdkUcbYWigBrs20dxpfPKkyix8mw


In [ ]:
!pip install flask flask_cors pyngrok


In [ ]:
!ngrok config add-authtoken 30ZJiOITXalIUigpHEzVbJJC81D_7Jx4484CDceXKxaav9HxZ


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import os
import json
import torch
from flask import Flask, request, jsonify
from flask_cors import CORS
from transformers import AutoProcessor, LlavaForConditionalGeneration, AutoModelForSeq2SeqLM, AutoTokenizer
from peft import PeftModel
import random

# --- CONFIG ---
API_KEY = "bnX1VvZn1v4XAkISdkUcbYWigBrs20dxpfPKkyix8mw"
MODEL_PATH = "/content/drive/MyDrive/Tyda/llava_finetune-VF"
DATASET_PATH = "/content/dataset_uni_final.jsonl"
NLLB_MODEL = "facebook/nllb-200-1.3B"
NLLB_LANG = "fra_Latn"

# --- LOAD MODELS ---
print("Chargement du modèle LLaVA...")
MODEL_NAME = "llava-hf/llava-1.5-7b-hf"
processor = AutoProcessor.from_pretrained(MODEL_NAME)
llava_model = LlavaForConditionalGeneration.from_pretrained(
    MODEL_NAME,
    load_in_4bit=True,
    torch_dtype=torch.float16,
    device_map="auto"
)
llava_model = PeftModel.from_pretrained(llava_model, MODEL_PATH)
llava_model.eval()

print("Chargement du traducteur NLLB...")
nllb_tokenizer = AutoTokenizer.from_pretrained(NLLB_MODEL)
nllb_model = AutoModelForSeq2SeqLM.from_pretrained(NLLB_MODEL).to("cuda" if torch.cuda.is_available() else "cpu")

print("Chargement du dataset...")
dataset = []
with open(DATASET_PATH, "r", encoding="utf-8") as f:
    for line in f:
        if line.strip():
            entry = json.loads(line)
            dataset.append(entry)

# --- FLASK APP ---
app = Flask(__name__)
CORS(app)

def filter_questions(category, level, sujet):
    filtered = []
    for q in dataset:
        if (not category or q.get("category", "").lower() == category.lower()) \
           and (not level or q.get("difficulty", "").lower() == level.lower()) \
           and (not sujet or sujet.lower() in q.get("sujet", "").lower()):
            filtered.append(q)
    return filtered

def llava_answer(question_text):
    prompt = f"Question: {question_text}\nAnswer:"
    inputs = processor(text=prompt, return_tensors="pt").to(llava_model.device)
    with torch.no_grad():
        outputs = llava_model.generate(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            max_new_tokens=50,
            do_sample=False,
            num_return_sequences=1,
            pad_token_id=processor.tokenizer.eos_token_id
        )
    generated = processor.tokenizer.decode(outputs[0], skip_special_tokens=True)
    generated = generated.replace(prompt, "").strip()
    return generated.split('\n')[0]

def llava_explanation(question_text, answer):
    prompt = f"Question: {question_text}\nAnswer: {answer}\nExplique pourquoi cette réponse est correcte."
    inputs = processor(text=prompt, return_tensors="pt").to(llava_model.device)
    with torch.no_grad():
        outputs = llava_model.generate(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            max_new_tokens=80,
            do_sample=False,
            num_return_sequences=1,
            pad_token_id=processor.tokenizer.eos_token_id
        )
    generated = processor.tokenizer.decode(outputs[0], skip_special_tokens=True)
    generated = generated.replace(prompt, "").strip()
    return generated

def translate_to_french(text):
    inputs = nllb_tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to(nllb_model.device)
    forced_bos_token_id = nllb_tokenizer.convert_tokens_to_ids(NLLB_LANG)
    translated_tokens = nllb_model.generate(**inputs, forced_bos_token_id=forced_bos_token_id, max_new_tokens=512)
    return nllb_tokenizer.decode(translated_tokens[0], skip_special_tokens=True)

@app.route('/generate_test', methods=['POST'])
def generate_test():
    if request.headers.get('Authorization') != f"Bearer {API_KEY}":
        return jsonify({'error': 'Unauthorized'}), 401
    data = request.json
    category = data.get('category', '')
    level = data.get('level', '')
    sujet = data.get('topic', '')
    num_questions = int(data.get('num_questions', 20))

    # 1. Filter dataset
    candidates = filter_questions(category, level, sujet)
    random.shuffle(candidates)
    selected = []
    for q in candidates:
        if len(selected) >= num_questions:
            break
        user_q = q["question"]
        correct_ans = q["answer"]
        model_ans = llava_answer(user_q)
        # Only keep if model gets it right (for MC: single char, for FR: substring match)
        if (len(correct_ans) == 1 and correct_ans.lower() in model_ans.lower()[:5]) or \
           (len(correct_ans) > 1 and correct_ans.lower() in model_ans.lower()):
            explanation = llava_explanation(user_q, correct_ans)
            selected.append({
                "question": user_q,
                "choices": q.get("choices", []),
                "answer": correct_ans,
                "explanation": explanation
            })

    # 2. Translate to French
    test_fr = []
    for q in selected:
        q_fr = {}
        q_fr["question"] = translate_to_french(q["question"])
        if q["choices"]:
            q_fr["choices"] = [translate_to_french(c) for c in q["choices"]]
        else:
            q_fr["choices"] = []
        q_fr["answer"] = q["answer"]  # Keep answer as is (for evaluation)
        q_fr["explanation"] = translate_to_french(q["explanation"])
        test_fr.append(q_fr)

    return jsonify({"questions": test_fr})

@app.route('/evaluate_test', methods=['POST'])
def evaluate_test():
    if request.headers.get('Authorization') != f"Bearer {API_KEY}":
        return jsonify({'error': 'Unauthorized'}), 401
    data = request.json
    questions = data.get('questions', [])
    user_answers = data.get('answers', [])
    results = []
    score = 0
    for idx, q in enumerate(questions):
        correct = q.get("answer", "").strip().lower()
        user = user_answers[idx].strip().lower() if idx < len(user_answers) else ""
        is_correct = (correct == user)
        if is_correct:
            score += 1
        results.append({
            "question": q.get("question"),
            "choices": q.get("choices", []),
            "user_answer": user,
            "correct_answer": correct,
            "is_correct": is_correct,
            "explanation": q.get("explanation")
        })
    return jsonify({
        "score": score,
        "total": len(questions),
        "results": results
    })

if __name__ == "__main__":
    # Optional: expose with ngrok in Colab
    try:
        from pyngrok import ngrok
        public_url = ngrok.connect(5000)
        print("🔗 Public API URL:", public_url)
    except Exception:
        print("ngrok not available or not installed.")
    app.run(host="0.0.0.0", port=5000)


Chargement du modèle LLaVA...


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Chargement du traducteur NLLB...
Chargement du dataset...
🔗 Public API URL: NgrokTunnel: "https://c1699fde5cac.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [30/Jul/2025 07:23:59] "POST /generate_test HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Jul/2025 07:25:36] "POST /evaluate_test HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Jul/2025 07:32:04] "POST /generate_test HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Jul/2025 07:34:20] "POST /evaluate_test HTTP/1.1" 200 -
